# 🚛 Araç Filosu Analizi
Bu notebook, atık toplama filosundaki araçların detaylı analizini içermektedir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Türkçe karakter desteği
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_palette("husl")
sns.set_style("whitegrid")

# Veri yükleme
fleet_df = pd.read_csv('fleet.csv')
truck_types_df = pd.read_csv('truck_types.csv')

print("Fleet Data Shape:", fleet_df.shape)
fleet_df.head(10)

## 📊 Veri Özeti

In [ ]:
print("=" * 60)
print("FLEET İSTATİSTİKLERİ")
print("=" * 60)
print(f"\nToplam Araç Sayısı: {len(fleet_df)}")
print(f"\nAraç Tipi Dağılımı:")
print(fleet_df['vehicle_type'].value_counts())
print(f"\n\nKapasite İstatistikleri:")
print(fleet_df[['capacity_m3', 'capacity_ton']].describe())

## 🎨 Araç Tipi Dağılımı

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Pie chart
vehicle_counts = fleet_df['vehicle_type'].value_counts()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
axes[0].pie(vehicle_counts.values, labels=vehicle_counts.index, autopct='%1.1f%%',
            startangle=90, colors=colors, textprops={'fontsize': 11})
axes[0].set_title('Araç Tipi Dağılımı', fontsize=14, fontweight='bold')

# Bar chart
vehicle_counts.plot(kind='barh', ax=axes[1], color=colors)
axes[1].set_xlabel('Araç Sayısı', fontsize=12)
axes[1].set_ylabel('Araç Tipi', fontsize=12)
axes[1].set_title('Araç Tiplerine Göre Dağılım', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# Değerleri çubukların üzerine yazdır
for i, v in enumerate(vehicle_counts.values):
    axes[1].text(v + 0.3, i, str(v), va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 📏 Kapasite Analizi

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Hacim kapasitesi kutu grafiği
fleet_df.boxplot(column='capacity_m3', by='vehicle_type', ax=axes[0, 0])
axes[0, 0].set_title('Araç Tiplerine Göre Hacim Kapasitesi', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Araç Tipi', fontsize=11)
axes[0, 0].set_ylabel('Kapasite (m³)', fontsize=11)
axes[0, 0].get_figure().suptitle('')  # Ana başlığı kaldır

# 2. Ton kapasitesi kutu grafiği
fleet_df.boxplot(column='capacity_ton', by='vehicle_type', ax=axes[0, 1])
axes[0, 1].set_title('Araç Tiplerine Göre Ton Kapasitesi', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Araç Tipi', fontsize=11)
axes[0, 1].set_ylabel('Kapasite (Ton)', fontsize=11)
axes[0, 1].get_figure().suptitle('')

# 3. Ton kapasitesi dağılımı (histogram)
for vehicle_type in fleet_df['vehicle_type'].unique():
    data = fleet_df[fleet_df['vehicle_type'] == vehicle_type]['capacity_ton']
    axes[1, 0].hist(data, alpha=0.6, label=vehicle_type, bins=10)
axes[1, 0].set_xlabel('Kapasite (Ton)', fontsize=11)
axes[1, 0].set_ylabel('Frekans', fontsize=11)
axes[1, 0].set_title('Ton Kapasitesi Dağılımı', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Scatter plot - Hacim vs Ton
for vehicle_type in fleet_df['vehicle_type'].unique():
    data = fleet_df[fleet_df['vehicle_type'] == vehicle_type]
    axes[1, 1].scatter(data['capacity_m3'], data['capacity_ton'], 
                      label=vehicle_type, s=100, alpha=0.6)
axes[1, 1].set_xlabel('Hacim Kapasitesi (m³)', fontsize=11)
axes[1, 1].set_ylabel('Ton Kapasitesi', fontsize=11)
axes[1, 1].set_title('Hacim vs Ton Kapasitesi İlişkisi', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 📈 Toplam Filo Kapasitesi

In [ ]:
# Araç tipine göre toplam kapasite
capacity_summary = fleet_df.groupby('vehicle_type').agg({
    'vehicle_id': 'count',
    'capacity_m3': 'sum',
    'capacity_ton': 'sum'
}).round(2)
capacity_summary.columns = ['Araç Sayısı', 'Toplam Hacim (m³)', 'Toplam Kapasite (Ton)']

print("\n" + "=" * 70)
print("FİLO KAPASİTE ÖZETİ")
print("=" * 70)
print(capacity_summary)
print("\nGENEL TOPLAM:")
print(f"Toplam Hacim Kapasitesi: {fleet_df['capacity_m3'].sum():.1f} m³")
print(f"Toplam Ton Kapasitesi: {fleet_df['capacity_ton'].sum():.1f} Ton")
print("=" * 70)

In [ ]:
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Toplam hacim kapasitesi
capacity_summary['Toplam Hacim (m³)'].plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Araç Tiplerine Göre Toplam Hacim Kapasitesi', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Toplam Kapasite (m³)', fontsize=11)
axes[0].set_xlabel('Araç Tipi', fontsize=11)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

for i, v in enumerate(capacity_summary['Toplam Hacim (m³)'].values):
    axes[0].text(i, v + 5, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

# Toplam ton kapasitesi
capacity_summary['Toplam Kapasite (Ton)'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Araç Tiplerine Göre Toplam Ton Kapasitesi', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Toplam Kapasite (Ton)', fontsize=11)
axes[1].set_xlabel('Araç Tipi', fontsize=11)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

for i, v in enumerate(capacity_summary['Toplam Kapasite (Ton)'].values):
    axes[1].text(i, v + 3, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 🎯 Temel Bulgular

1. **Filo Kompozisyonu:** Filodaki araç tipi dağılımı
2. **Kapasite Varyasyonu:** Ton kapasiteleri Gaussian dağılım gösteriyor
3. **Toplam Kapasite:** Filonun günlük taşıma kapasitesi
4. **Optimizasyon Potansiyeli:** Araç tiplerinin dengeli dağılımı

In [ ]:
# En son: Özet tablo
print("\n" + "=" * 70)
print("DETAYLI FİLO ÖZETİ")
print("=" * 70)
print(fleet_df.groupby('vehicle_type').agg({
    'vehicle_id': 'count',
    'capacity_m3': ['min', 'max', 'mean'],
    'capacity_ton': ['min', 'max', 'mean']
}).round(2))